In [1]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torchvision
from torch.utils import data
import numpy as np

In [2]:
mnist_train = torchvision.datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor()
)
mnist_test = torchvision.datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor()
)
print(mnist_train.data.shape)
print(mnist_test.data.shape)

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


In [3]:
# Note data is not shuffled
train_dataloader = DataLoader(mnist_train, batch_size=64)
test_dataloader = DataLoader(mnist_test, batch_size=64)

In [4]:
len(train_dataloader)

938

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, batch):
        flattened = self.flatten(batch)
        logits = self.linear_stack(flattened)
        return logits

model = MLP()
print(model)

MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
cross_entropy = nn.CrossEntropyLoss()
sgd = optim.SGD(model.parameters(), lr=5e-3, momentum=0.7)

In [8]:
def train(dataloader, model, loss_fn, optimizer, device):
    model.to(device)
    model.train()

    train_loss = 0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)

        loss = loss_fn(model(X), y)
        train_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Train loss: {train_loss / len(dataloader)}")

def test(dataloader, model, loss_fn, device):
    model.to(device)
    model.eval()

    test_loss, accurate = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            accurate += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    print(f"Test loss: {test_loss / len(dataloader)}, accuracy: {accurate / len(dataloader.dataset)}")

def train_loop(train_dataloader, test_dataloader, model, loss_fn, optimizer, device, epochs, test_every):
    for i in range(epochs):
        epoch = (i + 1)
        print(f"Epoch {epoch}\n----------------")
        train(train_dataloader, model, loss_fn, optimizer, device)
        if (epoch % test_every == 0):
            test(test_dataloader, model, loss_fn, device)
        print(f"----------------")

In [9]:
train_loop(train_dataloader, test_dataloader, model, cross_entropy, sgd, device, 50, 5)

Epoch 1
----------------
Train loss: 1.1346135139465332
----------------
Epoch 2
----------------
Train loss: 0.38175711035728455
----------------
Epoch 3
----------------
Train loss: 0.31366899609565735
----------------
Epoch 4
----------------
Train loss: 0.27615877985954285
----------------
Epoch 5
----------------
Train loss: 0.24707789719104767
Test loss: 0.2297335702534409, accuracy: 0.9337
----------------
Epoch 6
----------------
Train loss: 0.2225058674812317
----------------
Epoch 7
----------------
Train loss: 0.2013205885887146
----------------
Epoch 8
----------------
Train loss: 0.18301273882389069
----------------
Epoch 9
----------------
Train loss: 0.16713467240333557
----------------
Epoch 10
----------------
Train loss: 0.15335030853748322
Test loss: 0.14885480064530945, accuracy: 0.9544
----------------
Epoch 11
----------------
Train loss: 0.1413220912218094
----------------
Epoch 12
----------------
Train loss: 0.13069556653499603
----------------
Epoch 13
-------